# 02 - Data Analysis

Data Analysis on final DF

## Preliminaires 

### System 

In [ ]:
cd ../

In [ ]:
pwd

### Imports

In [ ]:
import os, sys, logging
from IPython.display import display, HTML

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

### Data

In [ ]:
df = pd.read_csv("data/final/final.csv")

## First Tour

### Display

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sample(10)

### Structure

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.nunique()

### Missing Values

In [ ]:
df

In [ ]:
df.isna().sum(axis=1)

In [ ]:
tmp = df.isna().sum(axis=1)
tmp[tmp == 0]

In [ ]:
df = df.loc[tmp[tmp == 0].index]
df

### Data Inspection

In [ ]:
df.describe().round(4)

In [ ]:
df = df.loc[df["country name"] != "Malta", :]
df

## EDA

### Numercials

In [ ]:
corr = df.select_dtypes(np.number).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(
    corr, mask=mask, cmap="coolwarm", vmin=-1, vmax=1, fmt=".2f", annot=True
)

In [ ]:
sns.pairplot(df, corner=True)

In [ ]:
sns.pairplot(df, corner=True, hue="uss30_region_name")

In [ ]:
for c in df.select_dtypes(np.number).columns:
    fig = px.box(df, x="uss30_region_name", y=c)
    fig.show()

### Clustering

In [ ]:
X_num = df.select_dtypes(np.number)
Xs = StandardScaler().fit_transform(X_num)
Xs = pd.DataFrame(Xs, columns=X_num.columns)
Xs.head()

In [ ]:
score_list = []
for k in range(2, 10):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(Xs)
    labels = kmeans.predict(Xs)
    score_list.append(
        {
            "k": k,
            "interia": kmeans.inertia_,
            "bd": davies_bouldin_score(Xs, labels),
            "silhouette": silhouette_score(Xs, labels),
        }
    )

In [ ]:
score_list = pd.DataFrame(score_list)
score_list

In [ ]:
for kpi in score_list.columns.to_list()[1:]:
    fig = px.line(score_list, x="k", y=kpi, title=kpi)
    fig.show()

In [ ]:
k = 6

kmeans = KMeans(n_clusters=k)
kmeans.fit(Xs)
labels = kmeans.predict(Xs)
df["cluster"] = labels
df.cluster = df.cluster.apply(lambda x: f"c_{x}")


df

In [ ]:
df.sort_values(by="cluster", inplace=True, ascending=True)

for label in df.cluster.unique().tolist():
    print(f"Cluster {label}")
    print("____________________")
    display(df.loc[df.cluster == label, :])
    display(df.loc[df.cluster == label, :].describe().round(4))

In [ ]:
for col in df.select_dtypes(np.number).columns.to_list():
    fig = px.box(df, x="cluster", y=col, title=col)
    fig.show()

### Maps

In [ ]:
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
world.head()

In [ ]:
eu_countries = df["country code"].values.tolist()
eu = world[world["iso_a3"].isin(eu_countries)]
eu.head()

In [ ]:
merged_data = eu.merge(df, left_on="iso_a3", right_on="country code")
merged_data.head()

In [ ]:
fig = px.choropleth(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color="cluster",
    color_continuous_scale="Viridis",
    scope="europe",
    # labels={"your_GDP_column": "GDP"},
)
fig.show()

In [ ]:
fig = px.choropleth(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color="ms.km2",
    color_continuous_scale="Viridis",
    scope="europe",
    labels={"your_GDP_column": "GDP"},
)
fig.show()